# 要約 
このJupyter Notebookでは、Kaggleの「LLM 20 Questions」コンペティションにおけるゲームデータの分析が行われています。具体的な問題としては、推測者と回答者のパフォーマンスを評価し、勝率や勝利回数を計算することが挙げられます。さらに、質問のワードクラウドや頻出キーワードの可視化も行われています。

### 問題に取り組む内容
主な目的は、ゲームのデータを基に各チームの勝率を分析し、推測者と回答者のリーダーボードを作成することです。これにより、どのチームやエージェントが最も成功しているかを把握し、戦略の改善に役立てることができます。また、質問内容のトレンド分析を通じて、どのような質問がよく使用されるかを視覚化しています。

### 使用されている手法とライブラリ
- **データ操作**: `pandas`を使用してCSVデータを読み込み、データフレームを操作。
- **自然言語処理**: `nltk`を利用して、ストップワードの管理やテキスト処理を行っています。
- **ワードクラウド生成**: `wordcloud`ライブラリを使用し、質問内容の視覚化を行い、人気のある質問を表示。
- **TF-IDF特徴量抽出**: `sklearn`の`TfidfVectorizer`を用いて、質問データの特徴量を分析することが可能です。
- **データ可視化**: `matplotlib`を利用して、生成したワードクラウドや解析結果を視覚的に表示。

このノートブックでは、データフレームの処理、ゲーム結果の集計、勝利回数の計算、リーダーボードの作成という一連のプロセスを通じて、参加エージェントのパフォーマンスを評価し、コンペティションの規模感や傾向を掴むことを目指しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の簡単な解説を列挙します。特に、実務を経験していないと馴染みのないものや、このノートブック特有のドメイン知識に焦点を当てています。

1. **リーダーボード (Leaderboard)**:
   コンペティションにおいて、参加者が提出したモデルやエージェントの性能を評価し、ランキング形式で表示するシステム。様々な指標で評価され、順位が決まる。

2. **ワードクラウド (Word Cloud)**:
   テキストデータ中の単語の出現頻度を視覚化する手法。頻出する単語が大きく表示されるため、データの特徴を直感的に理解しやすくなる。

3. **TF-IDF (Term Frequency-Inverse Document Frequency)**:
   テキストデータにおける重要度を示す指標。特定の単語が文書内でどれだけ頻繁に出現するか（Term Frequency）と、その単語が他の文書にどれだけ出現するかを考慮して重み付けされる。

4. **ストップワード (Stopwords)**:
   自然言語処理において、意味が薄いとされる一般的な単語（例: "the", "is", "in" 等）。これらの単語は分析から除外されることが多い。

5. **データフレーム (DataFrame)**:
   Pandasライブラリのデータ構造の一つで、ラベル付きの2次元データ（表形式）を持つ。行と列があり、効率的にデータ操作や分析が可能。

6. **リテラル評価 (Literal Evaluation)**:
   Pythonのデータ型（リストや辞書など）の文字列を、適切なPythonオブジェクトに変換する処理。`ast.literal_eval`関数を使用して安全に評価する。

7. **エピソード (Episode)**:
   特定の試合またはプレイのセッションを指す。コンペティションにおいては、特定のゲームの一連のプレイのことを意味する。

8. **正規化 (Normalization)**:
   データを一定の基準に従って変換すること。例えば、文字列を小文字に変換したり、不必要な空白を削除したりすることが含まれる。

9. **インターポレーション (Interpolation)**:
   画像を表示する際の手法の一つ。画像のピクセルを拡大または縮小する際に、ピクセルの間を補完して滑らかな表示を実現する方法。

10. **エージェント (Agent)**:
    機械学習やAIの文脈で用いられる用語で、自律的に行動し、環境からの入力に基づいて意思決定を行うシステムやプログラムを指す。

11. **カテゴリ (Category)**:
    データを特定のグループに分類するためのタグやラベル。コンペティションのドメインにおいて、特定のタイプや特性を持つデータを表現する。

12. **カウンター (Counter)**:
    Pythonの`collections`モジュールで提供される便利なクラスで、要素の出現頻度をカウントすることができる。テキストデータの分析に役立つ。

これらの用語について理解を深めることが、Jupyter Notebookの内容をよりスムーズに把握する手助けになるでしょう。

---


# 目次
各カテゴリーの最高のゲームを表示します:
* [推測者リーダーボード](#Guesser_leaderbord)
* [回答者リーダーボード](#Answerer_leaderbord)
* [推測者提出リーダーボード](#Guesser_submission_leaderbord)
* [回答者提出リーダーボード](#Answerer_submission_leaderbord)
* [質問ワードクラウド](#Questions_wordcould)

In [ ]:
import pandas as pd  # データ解析のためのライブラリをインポート
from ast import literal_eval  # リテラルを評価するための関数をインポート
from IPython.display import display, Markdown  # Jupyter Notebookで表示するためのモジュールをインポート
import string  # 文字列操作のためのモジュールをインポート
import json  # JSONデータを扱うためのモジュールをインポート

import nltk  # 自然言語処理のためのライブラリをインポート
from nltk.corpus import stopwords  # ストップワードを扱うためのモジュールをインポート
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator  # ワードクラウドを生成するライブラリをインポート
import matplotlib.pyplot as plt  # グラフ描画のためのライブラリをインポート
from sklearn.feature_extraction.text import TfidfVectorizer  # テキストデータのTF-IDF特徴量を抽出するためのモジュールをインポート
from collections import Counter  # 要素のカウントを行うためのモジュールをインポート

# 各ライブラリの役割:
# - pandas: データフレームを使ったデータ操作を容易にするライブラリ
# - ast: Pythonのリテラルを安全にPythonオブジェクトに変換するためのモジュール
# - IPython.display: Jupyter Notebook内でMarkdownやデータを表示するためのモジュール
# - string: 文字列操作用のストップワードや文字列定数を提供するモジュール
# - json: JSON形式のデータを操作するためのライブラリ
# - nltk: 自然言語処理のためのツールキット
# - wordcloud: ワードクラウドを作成するためのライブラリ
# - matplotlib: グラフを描画するためのオープンソースライブラリ
# - sklearn: 機械学習のためのライブラリ、テキストデータの特徴量抽出を含む
# - collections: データ集計のための便利なデータ構造を提供するモジュール

In [ ]:
df = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv", converters={col_list:literal_eval for col_list in ["answers", "questions", "guesses"]})
# CSVファイルを読み込みます。このファイルにはLLM 20 Questionsゲームのデータが含まれています。
# 'answers', 'questions', 'guesses'の各列は、文字列として保存されたリストを適切なリスト形式に変換するために、literal_evalを使用します。
# 変換後のデータフレームをdfという変数に格納します。
df  # データフレームの内容を表示します。

In [ ]:
def keyword_guessed(guess: str, keyword:str, alts:list[str]) -> bool:
    # 提出された推測がキーワードと一致するかどうかを判定する関数です。
    # 引数:
    # guess: ユーザーが推測した文字列
    # keyword: 正しいキーワード
    # alts: 代替のキーワードのリスト

    def normalize(s: str) -> str:
        # 文字列を正規化する内部関数です。
        # punctuation（句読点）を削除し、小文字に変換し、「the」を削除し、空白も削除します。
        t = str.maketrans("", "", string.punctuation)  # 句読点を削除するための変換テーブルを作成
        return s.lower().replace("the", "").replace(" ", "").translate(t)

    # ユーザーの推測とキーワードを正規化した後、比較します。
    if normalize(guess) == normalize(keyword):
        return True  # 推測がキーワードと一致する場合はTrueを返す

    # 代替キーワードのリストと比較します。
    for s in alts:
        if normalize(guess) == normalize(s):
            return True  # 推測が代替キーワードのいずれかと一致する場合もTrueを返す

    return False  # 推測がどれとも一致しない場合はFalseを返す

In [ ]:
exec(open("/kaggle/input/llm-20-questions/llm_20_questions/keywords.py").read()) # KEYWORDS_JSONを作成するためのスクリプトを実行します。
KEYWORDS_JSON = json.loads(KEYWORDS_JSON)  # 文字列形式のJSONデータをPythonの辞書型に変換します。

# keyword_guessed関数を適用するために、キーワードとその代替案を関連付ける辞書を作成します。
KEYWORD_TO_ALTS = {words["keyword"]:words["alts"] for category in KEYWORDS_JSON for words in category["words"]}
# KEYWORD_TO_ALTS辞書には、各キーワードに対してその代替キーワードのリストが格納されています。 
# 具体的には、各カテゴリーのすべてのキーワードと対応する代替案を含んでいます。
# KEYWORD_TO_ALTS  # このコメントアウトされた行を使うと、辞書の内容を表示できます。

In [ ]:
# `guessed`列を修正します。isinメソッドの代わりに競技用の関数を使用します。
df["guessed"] = df.apply(lambda row: keyword_guessed(row.guesses[-1] if len(row.guesses) > 0 else "", row.keyword, KEYWORD_TO_ALTS.get(row.keyword, [])), axis=1)
# 各行に対して、最後の推測がキーワードと一致しているかどうかを確認し、結果を`guessed`列に格納します。
# guessesが空でない場合は最後の推測を取得し、空の場合は空文字列を使用します。

# チームごとのゲーム数をカウントし、正規化するための辞書を作成します。
guesser_counts = {k: v for k, v in df.guesser.value_counts().items()}  # 推測者の名前とそのゲーム数の辞書
answerer_counts = {k: v for k, v in df.answerer.value_counts().items()}  # 回答者の名前とそのゲーム数の辞書

winner_df = df.loc[df.guessed]  # guessedがTrueの行を抽出し、勝者のデータフレームを作成します。
winner_df  # 勝者のデータフレームの内容を表示します。

In [ ]:
winner_df.guesser.value_counts().head(10)  # 勝者のデータフレームから、推測者の名前ごとの勝利回数をカウントし、上位10人を表示します。

In [ ]:
def fmt_team_winrate(team, team_fmt = "{:30} winrate: {:.3%} ({}/{})") -> str:
    # チームの勝率をフォーマットして返す関数です。
    # 引数:
    # team: チームの名前と勝利回数、総ゲーム数を含むタプル
    # team_fmt: フォーマットの文字列（デフォルトは勝率を表示するためのテンプレート）
    return team_fmt.format(team[0], team[1][0], team[1][1], team[1][2])
    # フォーマットした結果を返します。

def winrate(win_counts: dict, total_counts: dict) -> list:
    # 勝率を計算する関数です。
    # 引数:
    # win_counts: 勝利回数を含む辞書
    # total_counts: 各チームの総ゲーム数を含む辞書
    ratio_win = {k: (v / total_counts.get(k, 1), v, total_counts.get(k)) for k, v in win_counts.items()}
    # 各チームの勝率を計算し、勝利回数と総ゲーム数も取得して新しい辞書を作成します。
    
    return sorted(ratio_win.items(), key=lambda item: item[1][0], reverse=True)
    # 勝率でソートされた結果をリストとして返します。

In [ ]:
def print_winrate(winner_df=winner_df, df=df):
    # 勝率を表示する関数です。
    # 引数:
    # winner_df: 勝者データフレーム
    # df: 全ゲームデータフレーム
    nb_total_games = len(df)  # 全ゲーム数をカウントします。
    nb_wins = len(winner_df)  # 勝者データフレームの行数をカウントし、勝利回数を取得します。
    
    # フォーマットされた勝率を表示します。
    print(fmt_team_winrate(("Winrate total", (nb_wins / nb_total_games, nb_wins, nb_total_games))))

print_winrate()  # 勝率を計算して表示する関数を実行します。

# 推測者リーダーボード

In [ ]:
guesser_win = winrate(winner_df.guesser.value_counts().items(), guesser_counts)  # 推測者の勝率を計算します。
for team in guesser_win[:10]:  # 上位10チームについて
    print(fmt_team_winrate(team))  # フォーマットされた勝率を表示します。

# 回答者リーダーボード
[目次](#Table_of_Contents)

In [ ]:
answerer_win = winrate(winner_df.answerer.value_counts().items(), answerer_counts)  # 回答者の勝率を計算します。
for team in answerer_win[:10]:  # 上位10チームについて
    print(fmt_team_winrate(team))  # フォーマットされた勝率を表示します。

## 最も推測されたキーワード:

In [ ]:
winner_df.keyword.value_counts()  # キーワードの出現回数をカウントします。フランスが上位にあるのは、おそらくベースラインの例に起因しています。

In [ ]:
episode_agents = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/EpisodeAgents.csv")  # EpisodeAgents.csvファイルを読み込みます。
episode_agents = episode_agents.groupby(["EpisodeId", "SubmissionId"]).first()  # EpisodeIdとSubmissionIdでグループ化し、最初の行を取得します。
episode_agents  # グループ化されたデータの内容を表示します。

In [ ]:
def show_game(row: pd.Series):
    # ゲームデータを表示する関数です。
    # 引数:
    # row: ゲームに関する情報を含むPandasのSeries

    if row.empty:  # 行が空の場合は何もしません。
        return 
    
    table_format = "| {question} | {answer} | {guess} |"  # テーブルフォーマットを定義します。
    
    # ゲームの質問、回答、推測の履歴を作成します。
    history = "\n".join([table_format.format(question="question", answer="answer", guess="guess"), 
                         table_format.format(question="---", answer="---", guess="---")] + 
                        [table_format.format(question=question, answer=answer, guess=guess) 
                         for answer, question, guess in zip(row.answers, row.questions, row.guesses)])
    
    # 推測者のスコアを取得します。
    guesser_score = episode_agents.loc[(row.game_num, row.guesser_SubmissionId)]
    guesser_score = f"{guesser_score.InitialScore:.2f}->{guesser_score.UpdatedScore:.2f} Confidence={guesser_score.UpdatedConfidence:.2f}" if not guesser_score.empty else ""
    
    # 回答者のスコアを取得します。
    answerer_score = episode_agents.loc[(row.game_num, row.answerer_SubmissionId)]
    answerer_score = f"{answerer_score.InitialScore:.2f}->{answerer_score.UpdatedScore:.2f} Confidence={answerer_score.UpdatedConfidence:.2f}" if not answerer_score.empty else ""
    
    # ゲーム情報をMarkdown形式で表示します。
    return display(Markdown(f"""\
* {row.game_num=}
* Guesser:  **{row.guesser}** score: {guesser_score}
* Answerer: **{row.answerer}** score: {answerer_score}
* Keyword: **{row.keyword}** Category: {row.category} 
* Nb round: {row.nb_round} {row.CreateTime}

{history}
"""))
    
show_game(winner_df.iloc[0])  # 最初の勝者のゲームを表示します。この例では「答えはフランス」という推測がゲームに勝てなかったことを示します。正確なキーワードまたは代替キーワードが必要です。

In [ ]:
display(Markdown(f"## 最優秀回答者 **{fmt_team_winrate(answerer_win[0])}**"))  # 最も優れた回答者を表示します。
for i, row in winner_df[winner_df.answerer == answerer_win[0][0]].iterrows():  # 最優秀回答者が勝利した全ゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。

In [ ]:
display(Markdown(f"## 最優秀推測者 **{fmt_team_winrate(guesser_win[0])}**"))  # 最も優れた推測者を表示します。
for i, row in winner_df[winner_df.guesser == guesser_win[0][0]].iterrows():  # 最優秀推測者が勝利した全ゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。

# 最優秀提出物
同様に、推測者や回答者ではなく、SubmissionIdごとに表示します。  
目的は、より正確なリーダーボードを持つことです。チームは異なる戦略や結果を持つ複数のエージェントを提出することができます。  
SubmissionIdを使用することで、新しい提出物がうまく機能するかどうかを確認できます（チームが何回提出したかや、何ゲームプレイしたかに関係なく）。

In [ ]:
guesser_sub_counts = {k: v for k, v in df.guesser_SubmissionId.value_counts().items()}  # 推測者のSubmissionIdごとのゲーム数をカウントします。
answerer_sub_counts = {k: v for k, v in df.answerer_SubmissionId.value_counts().items()}  # 回答者のSubmissionIdごとのゲーム数をカウントします。

guesser_sub_win = winrate(winner_df.guesser_SubmissionId.value_counts().items(), guesser_sub_counts)  # 推測者のSubmissionIdごとの勝率を計算します。
answerer_sub_win = winrate(winner_df.answerer_SubmissionId.value_counts().items(), answerer_sub_counts)  # 回答者のSubmissionIdごとの勝率を計算します。

# SubmissionIdとそれに対応するチーム名の辞書を作成します。
subId_to_team = {**winner_df.groupby("answerer_SubmissionId").answerer.first(), 
                 **winner_df.groupby("guesser_SubmissionId").guesser.first()}  # 回答者と推測者のSubmissionIdごとに最初のチームを取得します。

# 推測者提出リーダーボード
[目次](#Table_of_Contents)

In [ ]:
for team in guesser_sub_win[:10]:  # 上位10チームについて
    team = (f"{subId_to_team[team[0]]} {team[0]}", (team[1]))  # SubmissionIdとチーム名を組み合わせます。
    print(fmt_team_winrate(team))  # フォーマットされた勝率を表示します。

In [ ]:
display(Markdown(f"## 最優秀推測者エージェントのゲーム: SubmissionId={fmt_team_winrate(guesser_sub_win[0])}"))  # 最も優れた推測者エージェントのゲームを表示します。
for i, row in winner_df[winner_df.guesser_SubmissionId == guesser_sub_win[0][0]].iterrows():  # 最優秀推測者のSubmissionIdに基づいてゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。

# 回答者提出リーダーボード
[目次](#Table_of_Contents)

In [ ]:
for team in answerer_sub_win[:10]:  # 上位10チームについて
    team = (f"{subId_to_team[team[0]]} {team[0]}", (team[1]))  # SubmissionIdとチーム名を組み合わせます。
    print(fmt_team_winrate(team))  # フォーマットされた勝率を表示します。

In [ ]:
display(Markdown(f"## 最優秀回答者エージェントのゲーム: {subId_to_team[answerer_sub_win[0][0]]} SubmissionId={fmt_team_winrate(answerer_sub_win[0])}"))  # 最も優れた回答者エージェントのゲームを表示します。
for i, row in winner_df[winner_df.answerer_SubmissionId == answerer_sub_win[0][0]].iterrows():  # 最優秀回答者のSubmissionIdに基づいてゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。

In [ ]:
display(Markdown(f"## 2番目に優れた回答者エージェントのゲーム: {subId_to_team[answerer_sub_win[1][0]]} SubmissionId={fmt_team_winrate(answerer_sub_win[1])}"))  # 2番目に優れた回答者エージェントのゲームを表示します。
for i, row in winner_df[winner_df.answerer_SubmissionId == answerer_sub_win[1][0]].iterrows():  # 2番目に優れた回答者のSubmissionIdに基づいてゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。

# 最良スコアを持つ提出物からのゲーム
以前の提出物もカウントしますが、今のところあまり役に立ちません。

In [ ]:
_, best_subId = episode_agents.iloc[episode_agents.UpdatedScore.argmax()].name  # 最良スコアを持つSubmissionIdを取得します。
display(Markdown(f"## 最良スコアを持つ提出物: {subId_to_team[best_subId]} SubmissionId={best_subId}"))  # 最良スコアを持つ提出物を表示します。
for i, row in winner_df[winner_df.answerer_SubmissionId == best_subId].iterrows():  # 最良スコアを持つ回答者のゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。
    
for i, row in winner_df[winner_df.guesser_SubmissionId == best_subId].iterrows():  # 最良スコアを持つ推測者のゲームを繰り返し処理します。
    show_game(row)  # 各ゲームの詳細を表示します。

# 質問ワードクラウド

In [ ]:
t = str.maketrans("", "", string.punctuation)  # 句読点を削除するための変換テーブルを作成します。
questions_flat = [question.lower() for questions in df.questions.values for question in questions]  # 質問を小文字に変換しフラット化します。
print("ユニークな質問の数:", len(set(questions_flat)))  # ユニークな質問の数を表示します。

In [ ]:
Counter(questions_flat).most_common(50)  # 最も一般的な50の質問をカウントし、表示します。

In [ ]:
nltk.download('stopwords')  # NLTKのストップワードデータセットをダウンロードします。
stop_words = list(stopwords.words('english'))  # 英語のストップワードのリストを取得します。
stop_words  # ストップワードのリストを表示します。

In [ ]:
%%time  # このセルの実行時間を計測します。
top_q_str = " ".join(question for question, count in Counter(questions_flat).most_common(200))  # 最も一般的な200の質問を結合して文字列を作成します。
wordcloud = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(top_q_str)  # ワードクラウドを生成します。

# 生成した画像を表示します。
plt.figure(figsize=(30, 20))  # 図のサイズを設定します。
plt.imshow(wordcloud, interpolation='bilinear')  # ワードクラウドを表示します。
plt.axis("off")  # 軸を非表示にします。
plt.show()  # 図を表示します。

In [ ]:
%%time  # このセルの実行時間を計測します。
stop_words += ["keyword", "question", "questions", "sure", "please", "answer", "answerer", "answers", "yes"]  # 追加のストップワードをリストに加えます。
wordcloud_words = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(top_q_str)  # 新しいストップワードリストを使用してワードクラウドを生成します。

# 生成した画像を表示します。
plt.figure(figsize=(30, 20))  # 図のサイズを設定します。
plt.imshow(wordcloud_words, interpolation='bilinear')  # ワードクラウドを表示します。
plt.axis("off")  # 軸を非表示にします。
plt.show()  # 図を表示します。

In [ ]:
# tfidf = TfidfVectorizer(max_df=.98, min_df=.01, stop_words=stop_words, use_idf=True, norm=None)  # TF-IDFベクトライザーを設定します。文書の98%に出現する単語や1%未満の単語を除外します。
# questions_tfidf = tfidf.fit_transform(questions_flat)  # 質問データをTF-IDF行列に変換します。
# questions_tfidf  # TF-IDF行列を表示します。

In [ ]:
# tfidf.get_feature_names_out()  # TF-IDFベクトライザーによって得られる特徴（単語）の名前を出力します。

In [ ]:
# questions_tfidf = pd.DataFrame(questions_tfidf.todense(), columns=tfidf.get_feature_names_out())  # TF-IDF行列をDense形式のデータフレームに変換し、特徴名を列名として設定します。
# questions_tfidf  # 作成したデータフレームを表示します。

# 勝利したゲームから最も多く尋ねられた質問

In [ ]:
# questions_win = {question for questions in winner_df.questions.values for question in questions}  # 勝利したゲームの質問をセットとして取得します。
# print(len(questions_win))  # ユニークな勝利したゲームの質問の数を表示します。
# questions_win  # 収集した質問のセットを表示します。

In [ ]:
all_questions_win = [question.lower() for questions in winner_df.questions.values for question in questions]  # 勝利したゲームの質問を小文字に変換してリストに全て取得します。
len(all_questions_win)  # 勝利したゲームの質問の総数を表示します。

In [ ]:
Counter(all_questions_win).most_common(50)  # 勝利したゲームからの質問の中で最も一般的な50の質問をカウントし、表示します。

In [ ]:
%%time  # このセルの実行時間を計測します。
stop_words += ["keyword", "question", "questions", "sure", "please", "answer", "answerer", "answers", "yes"]  # 追加のストップワードをリストに加えます。
wordcloud_words = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(" ".join(question for question, count in Counter(all_questions_win).most_common(50)))  # 最も一般的な50の質問を使ってワードクラウドを生成します。

# 生成した画像を表示します。
plt.figure(figsize=(30, 20))  # 図のサイズを設定します。
plt.imshow(wordcloud_words, interpolation='bilinear')  # ワードクラウドを表示します。
plt.axis("off")  # 軸を非表示にします。
plt.show()  # 図を表示します。

# キーワードの大文字が成功したエピソード

In [ ]:
winner_df_maj = winner_df.loc[winner_df.category.isin(["place", "things"])]  # カテゴリーが「place」と「things」の勝者データフレームを抽出します。
winner_df_maj  # 抽出したデータフレームを表示します。

In [ ]:
winner_df_maj.answerer.value_counts()  # カテゴリーが「place」と「things」の勝者における回答者の勝利回数をカウントし、表示します。

In [ ]:
winner_df_maj.guesser.value_counts()  # カテゴリーが「place」と「things」の勝者における推測者の勝利回数をカウントし、表示します。

# コメント

> ## Divyanshu
> 
> 素晴らしいノートブックです。票を入れました。
> 
> 私の作業を確認してフィードバックをいただけますか？良ければ票を入れてください。
> 
> 

---

> ## OminousDude
> 
> とても役に立ちます、あなたには票を入れる価値があります！
> 
> 

---

> ## Staratnyte
> 
> 素晴らしい仕事です！とても役立つ作業です。
> 
> 

---

